# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import pprint

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
file_path_list = []
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list.extend(glob.glob(os.path.join(root, '*')))

pprint.pprint(file_path_list)


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('event_datafile_new.csv')

df

In [ ]:
# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [ ]:
## Begin writing your Apache Cassandra code in the cells below

In [ ]:
#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
# You will need to install and run a Cassandra Server on your machine

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 

from cassandra.query import SimpleStatement

keyspace_name = "Sparkify"
replication_options = {
    'class': 'SimpleStrategy',
    'replication_factor': 1
}

create_keyspace_query = f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH replication = {replication_options};"
session.execute(SimpleStatement(create_keyspace_query))

In [ ]:
# Checking if Keyspace was created

query = "SELECT keyspace_name FROM system_schema.keyspaces;"
rows = session.execute(query)

# Print existing keyspaces
for row in rows:
    print(row.keyspace_name)

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above
keyspace_name = "sparkify"
session.set_keyspace(keyspace_name)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
"""Creating tables for query 1: For the music_history table, 
    we chose the sessionId as the partition key because it allows us
    to distribute the data evenly across the cluster and
    group all the records that belong to the same session on the same node. 
    Then, we use itemInSession as the clustering key to sort the 
    records within each partition according to the order in which 
    the songs were played during the session."""

query = """
    CREATE TABLE IF NOT EXISTS sparkify.music_history (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession)
    )
"""

session.execute(query)

In [ ]:
# Creating tables for query 2

"""Creating tables for query 2: For the music_history_by_user table, 
    we chose userId and sessionId as the composite partition key
    because it allows us to group all the records that belong to the same user
    and session together on the same node. 
    Then, we use itemInSession as the clustering key to sort the records 
    within each partition according to the order in which the songs 
    were played during the session. 
    We also include firstName and lastName as additional columns
    to store the user's name for each record."""

query = """
    CREATE TABLE IF NOT EXISTS sparkify.music_history_by_user (
    userId INT,
    sessionId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY ((userId, sessionId), itemInSession)
) WITH CLUSTERING ORDER BY (itemInSession ASC);
"""

session.execute(query)

In [ ]:
"""Creating tables for query 3: For the music_history_by_song table,
    we chose song as the partition key because it allows us to group all
    the records that belong to the same song together on the same node. 
    Then, we use userId as the clustering key to sort the records
    within each partition according to the order in which the users
    listened to the song."""

query = """
    CREATE TABLE IF NOT EXISTS music_history_by_song (
    song text,
    userId int,
    firstName text,
    lastName text,
    PRIMARY KEY (song, userId)
);
"""

session.execute(query)

In [ ]:
# Retrieve metadata about the keyspace
keyspace_metadata = cluster.metadata.keyspaces['sparkify']

# Print column names for each table in the keyspace
for table in keyspace_metadata.tables.values():
    print(f"Table: {table.name}")
    for column in table.columns.values():
        print(f"Column: {column.name}")

## QUERY 1

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history (sessionId,itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (
            int(line[8]),  # sessionId
            int(line[3]),  # itemInSession
            line[0],       # artist
            line[9],       # song
            float(line[5]) # length
        ))

In [ ]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

rows = session.execute("SELECT artist, song, length FROM music_history WHERE sessionId = 338 AND itemInSession = 4;")
for row in rows:
    print(row.artist, row.song, row.length)

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

rows = session.execute("SELECT * FROM music_history LIMIT 10")
for row in rows:
    print(row)

In [ ]:
## QUERY 2

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (
            int(line[10]),  # userId
            int(line[8]),   # sessionId
            int(line[3]),   # itemInSession
            line[0],        # artist
            line[9],        # song
            line[1],        # firstName
            line[4],        # lastName
        ))

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

rows = session.execute("SELECT artist, song, firstname, lastname FROM sparkify.music_history_by_user WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession ASC")
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

rows = session.execute("SELECT * FROM music_history_by_user LIMIT 10")
for row in rows:
    print(row)

In [ ]:
## QUERY 3

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statement to the `query` variable
        query = "INSERT INTO music_history_by_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (
            line[9],        # song
            int(line[10]),  # userId
            line[1],        # firstName
            line[4],        # lastName
        ))

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


rows = session.execute("SELECT firstName, lastName FROM music_history_by_song WHERE song = 'All Hands Against His Own';")
for row in rows:
    print(row.firstname, row.lastname) 

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

rows = session.execute("SELECT * FROM music_history_by_song LIMIT 10")
for row in rows:
    print(row)

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

session.execute("DROP TABLE IF EXISTS music_history")
session.execute("DROP TABLE IF EXISTS music_history_by_user")
session.execute("DROP TABLE IF EXISTS music_history_by_song")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()